In [0]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = ('drive/My Drive/ML Kaggle Group')

In [0]:
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import numpy as np

In [5]:
train = pd.read_csv("/content/drive/My Drive/ML Kaglle Group/tcd-ml-1920-group-income-train.csv",index_col=None) 
sub_test = pd.read_csv("/content/drive/My Drive/ML Kaglle Group/tcd-ml-1920-group-income-test.csv",index_col=None)
submission = pd.read_csv("/content/drive/My Drive/ML Kaglle Group/tcd-ml-1920-group-income-submission.csv",index_col=None)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data = pd.concat([train,sub_test],ignore_index=True)

In [0]:
data = data.replace('nA',np.nan)

In [0]:
def process_satisfaction(dataset):
  dataset['Satisfation with employer'].replace(np.nan, 'Missing', inplace=True)
def process_exp(dataset):
  median = dataset[dataset['Work Experience in Current Job [years]']!='#NUM!']['Work Experience in Current Job [years]'].median()
  dataset['Work Experience in Current Job [years]'].replace('#NUM!', median, inplace=True)
def process_housing(dataset):
  dataset['Housing Situation'].replace('0', 'Missing', inplace=True)
  dataset['Housing Situation'].replace(np.nan, 'Missing', inplace=True)

def process_hair_color(dataset):
    dataset['Hair Color'].replace('0', 'Missing', inplace=True)
    dataset['Hair Color'].replace('unknown', 'Missing', inplace=True)
    dataset['Hair Color'].replace(np.nan, 'Missing', inplace=True)


def process_university_degree(dataset):
    dataset['University Degree'].replace(np.nan, 'Missing', inplace=True)


def process_profession(dataset):
    dataset['Profession'].replace(np.nan, 'Missing', inplace=True)


def process_year_of_record(dataset):
    year_median = dataset['Year of Record'].median()
    dataset['Year of Record'].replace(np.nan, year_median, inplace=True)


def process_age(dataset):
    age_median = dataset['Age'].median()
    dataset['Age'].replace(np.nan, age_median, inplace=True)
    dataset['Age'] = (dataset['Age'] * dataset['Age']) ** (0.5)


def process_gender(dataset):
    dataset['Gender'].replace('f', 'female', inplace=True)
    dataset['Gender'].replace('0', 'Missing', inplace=True)
    dataset['Gender'].replace('other', 'Missing', inplace=True)
    dataset['Gender'].replace('unknown', 'Missing', inplace=True)
    dataset['Gender'].replace(np.nan, 'Missing', inplace=True)

In [0]:
def preprocess_dataset(dataset):
    process_gender(dataset)
    process_age(dataset)
    process_year_of_record(dataset)
    process_profession(dataset)
    process_university_degree(dataset)
    process_hair_color(dataset)
    process_housing(dataset)
    process_satisfaction(dataset)
    process_exp(dataset)
    return dataset

In [0]:
data_processed = preprocess_dataset(data)

In [11]:
data_processed.dtypes

Instance                                                      int64
Year of Record                                              float64
Housing Situation                                            object
Crime Level in the City of Employement                        int64
Work Experience in Current Job [years]                       object
Satisfation with employer                                    object
Gender                                                       object
Age                                                         float64
Country                                                      object
Size of City                                                  int64
Profession                                                   object
University Degree                                            object
Wears Glasses                                                 int64
Hair Color                                                   object
Body Height [cm]                                

In [0]:
def convert_to_int(dataset,col):
  dataset[col] = dataset[col].astype(float)
  return dataset

In [0]:
data_processed['Yearly Income in addition to Salary (e.g. Rental Income)']= data_processed['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda st:st.split(' ')[0])
data_processed = convert_to_int(data_processed,'Yearly Income in addition to Salary (e.g. Rental Income)')
data_processed = convert_to_int(data_processed,'Work Experience in Current Job [years]')



In [0]:
data_processed = convert_to_int(data_processed,'Total Yearly Income [EUR]')

In [0]:
def get_mean_rank(df,col):
  df1 = df.groupby([col])['Total Yearly Income [EUR]'].mean().sort_values()
  dict_rank = {}
  counter = 1
  for row in  df1.iteritems():
    dict_rank[row[0]] = counter
    counter+=1
  return dict_rank

In [0]:
dict_profession = get_mean_rank(data_processed,'Profession')

In [0]:
dict_year = get_mean_rank(data_processed,'Year of Record')

In [0]:
dict_country = get_mean_rank(data_processed,"Country")

In [0]:
from sklearn.preprocessing import LabelEncoder
for col in data_processed.dtypes[data_processed.dtypes == 'object'].index.tolist():
    if col in ['Profession','Country']:
      continue
    else:
      feat_le = LabelEncoder()
      feat_le.fit(data_processed[col].unique().astype(str))
      data_processed[col] = feat_le.transform(data_processed[col].astype(str))

In [0]:
data_processed['Profession'] = data_processed['Profession'].map(dict_profession)
data_processed['Year of Record'] = data_processed['Year of Record'].map(dict_year)
data_processed['Country'] = data_processed['Country'].map(dict_country)


In [48]:
data_processed.columns

Index(['Instance', 'Year of Record', 'Housing Situation',
       'Crime Level in the City of Employement',
       'Work Experience in Current Job [years]', 'Satisfation with employer',
       'Gender', 'Age', 'Country', 'Size of City', 'Profession',
       'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]',
       'Yearly Income in addition to Salary (e.g. Rental Income)',
       'Total Yearly Income [EUR]'],
      dtype='object')

In [53]:
from sklearn.model_selection import train_test_split
X_train,X_test = data_processed.iloc[:1048574],data_processed.iloc[1048574:]
X_train['Total Yearly Income [EUR]'] = X_train['Total Yearly Income [EUR]'] - X_train['Yearly Income in addition to Salary (e.g. Rental Income)']
Y_train = data_processed['Total Yearly Income [EUR]'].iloc[:1048574]
X_test_id = data_processed['Instance'].iloc[1048574:]
train_rental = X_train['Yearly Income in addition to Salary (e.g. Rental Income)']
test_rental = X_test['Yearly Income in addition to Salary (e.g. Rental Income)']
X_train.drop(['Total Yearly Income [EUR]','Instance','Yearly Income in addition to Salary (e.g. Rental Income)'],axis=1,inplace=True)
X_test.drop(['Total Yearly Income [EUR]','Instance','Yearly Income in addition to Salary (e.g. Rental Income)'],axis=1,inplace=True)
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=1234)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
y_train = np.log(y_train)
y_val = np.log(y_val)

In [0]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

##### 7. transform y ###############
y_train_log = np.log(y_train)
###################################

###### 8. fit model and predict the income ###############
regressor.fit(x_train, y_train_log)
y_pred_lin = np.exp(regressor.predict(x_val))

In [55]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val,y_pred_lin)

24847.547687668834

In [56]:
import lightgbm as lgb
model_lgb = lgb.LGBMRegressor(objective='regression_l1',boosting="gbdt", num_leaves=70, n_estimators=1500)
model_lgb.fit(x_train, y_train_log)

LGBMRegressor(boosting='gbdt', boosting_type='gbdt', class_weight=None,
              colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1500, n_jobs=-1, num_leaves=70,
              objective='regression_l1', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [0]:
y_pred = np.exp(model_lgb.predict(x_val))

In [58]:
mean_absolute_error(y_val,y_pred)

14720.62582294973

In [59]:
import lightgbm as lgb
params = {
          'max_depth': 20,
          'learning_rate': 0.01,
          "boosting": "gbdt",
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1
         }
trn_data = lgb.Dataset(x_train, label=y_train_log)
val_data = lgb.Dataset(x_val, label=np.log(y_val))
# test_data = lgb.Dataset(X_test)
clf = lgb.train(params, trn_data, 50000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)


Training until validation scores don't improve for 500 rounds.
[1000]	training's l1: 0.356589	valid_1's l1: 0.359675
[2000]	training's l1: 0.34624	valid_1's l1: 0.350318
[3000]	training's l1: 0.340568	valid_1's l1: 0.345533
[4000]	training's l1: 0.336407	valid_1's l1: 0.342237
[5000]	training's l1: 0.333402	valid_1's l1: 0.340176
[6000]	training's l1: 0.330826	valid_1's l1: 0.338474
[7000]	training's l1: 0.328626	valid_1's l1: 0.337129
[8000]	training's l1: 0.326551	valid_1's l1: 0.335908
[9000]	training's l1: 0.324667	valid_1's l1: 0.334883
[10000]	training's l1: 0.323057	valid_1's l1: 0.334147
[11000]	training's l1: 0.321535	valid_1's l1: 0.333493
[12000]	training's l1: 0.320079	valid_1's l1: 0.332867
[13000]	training's l1: 0.318617	valid_1's l1: 0.332237
[14000]	training's l1: 0.317244	valid_1's l1: 0.331715
[15000]	training's l1: 0.315957	valid_1's l1: 0.331315
[16000]	training's l1: 0.314729	valid_1's l1: 0.330992
[17000]	training's l1: 0.313442	valid_1's l1: 0.330559
[18000]	trai

In [0]:
lgb_pred = np.exp(clf.predict(x_val))

In [61]:
mean_absolute_error(lgb_pred,y_val)

14396.914003103377

In [0]:
predict = clf.predict(X_test)

In [0]:

parameters = {'nthread':[4], 
              'objective':['reg:squarederror'],
                   'min_child_weight': [5],
        'gamma': [0.5],
        'subsample': [0.8],
        'colsample_bytree': [1.0],
        'max_depth': [11]}

In [0]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
x_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train.columns.values]


In [0]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
x_val.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns.values]


In [0]:
from sklearn.preprocessing import LabelEncoder
import xgboost
from sklearn.metrics import mean_squared_error
xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=3000, nthread=6, scale_pos_weight=1, seed=27,tree_method='gpu_hist')

In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_absolute_error')
gsearch1.fit(x_train,y_train_log)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed: 39.7min remaining:    0.0s
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed: 39.7min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=3000, n_jobs=1, nthread=6,
                                    objective='reg:linear', random_state...
                                    scale_pos_weight=1, seed=27, silent=None,
                                    subsample=1, tree_method='gpu_hist',
                                    verbosity=1),
             iid=False, n_jobs=6,
             param_grid={'colsample_bytree': [1.0], 'gamma': [0.5],
                         'max_depth': [11], 'min_child_weigh

In [0]:
xgb_pred = np.exp(gsearch1.predict(x_val))

In [75]:
mean_absolute_error(xgb_pred,y_val)

14737.00556075361

In [0]:
ensembled_prediction = (0.45*xgb_pred+0.55*lgb_pred)

In [77]:
mean_absolute_error(ensembled_prediction,y_val)

14148.898693296702

In [0]:
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

In [0]:
xg_sub_pred = np.exp(gsearch1.predict( X_test))

In [0]:
ensembled_prediction = (0.40*xg_sub_pred+0.60*np.exp(predict))

In [92]:
sub_df = pd.DataFrame({'Instance':X_test_id,
                       'Total Yearly Income [EUR]':np.exp(predict)+test_rental})
sub_df.head()

,Instance,Total Yearly Income [EUR]
1048574,1,36424.623770
1048575,2,8775.260883
1048576,3,5535.891094
1048577,4,81406.859081
1048578,5,5388.320392


In [0]:
sub_df.to_csv('sub_finallgb.csv')

In [0]:
xgb_thresh = [x for x in np.arange(0, 1, 0.05)]
lgb_thresh = [x for x in np.arange(0, 1, 0.05)]

In [86]:
score = 10000000
best_xgb = 0
best_lgb = 0
for x in xgb_thresh:
  for y in lgb_thresh:
    ensembled_prediction = (x*xgb_pred+y*lgb_pred)
    ensembeled_error = mean_absolute_error(y_val, ensembled_prediction)
    if ensembeled_error < score:
      score = ensembeled_error
      best_xgb = x
      best_lgb = y
      print(score)

68451.80180163555
65255.405982653785
62059.03063731531
58862.7258388988
55666.62627208734
52471.11225800706
49276.79715080375
46085.1440365833
42897.379437104486
39715.01584366368
36542.530260629435
33385.03045241418
30253.12915789074
27166.57890275745
24165.393482915297
21332.822952636798
18800.026698987815
16733.03884807694
15269.368793775766
14483.497537550455
14413.283099397906
14333.252471638538
14279.454156721515
14234.833171803077
14199.147419700757
14171.958943170384
14153.501012652325
14143.829954442685
14142.572940572996


In [94]:
x_val['Income'] = lgb_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
x_val['true_income'] = y_val

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
train.loc[110509]

Instance                                                            110510
Year of Record                                                        1948
Housing Situation                                                        0
Crime Level in the City of Employement                                 174
Work Experience in Current Job [years]                                  22
Satisfation with employer                                   Somewhat Happy
Gender                                                             unknown
Age                                                                     56
Country                                                           Honduras
Size of City                                                       1157491
Profession                                                          packer
University Degree                                                       No
Wears Glasses                                                            0
Hair Color               

In [0]:
li = []

In [0]:
li2 = []

In [0]:
for row in x_val.iterrows():
  if (row[1]['true_income']-row[1]['Income']) >15000:
    li2.append(row[1]['true_income']-row[1]['Income'])


In [124]:
len(x_val)

209715

In [0]:
df2 = pd.DataFrame(columns=train.columns)

In [139]:
len(li2)

29432

In [143]:
df2['diff'] = li2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df5= df2.groupby(['Profession'])['diff'].mean().sort_values()

In [0]:
Professions = ['Audit Clerk'                       
'budget review specialist'          
'Baggage Handler'                  
'accountable project manager'       
'deputy budget director','ticket taker','crime analyst','timekeeper','desktop publisher','emergency field logistics coordinator','Baker']

In [188]:
df5[1180:]

Profession
actuary                                 132942.415822
wireless coordinator                    133310.997691
child welfare analyst                   138027.623393
buyer                                   140722.065199
architectural designer                  142508.533816
clinical director                       142689.709943
city medical examiner                   144348.986909
artist                                  144681.539003
civil engineer                          147166.272694
criminal investigator                   151200.717307
tracking and monitoring data analyst    155755.657475
boiler inspector                        170116.136343
Balloonist                              172625.153948
Baker                                   187922.965196
dressmaker                              201509.772518
architect                               201847.857554
Biologist                               212121.587295
Brewer                                  229472.354369
Audit Clerk      

In [0]:
years = [2011:2019]

In [0]:
Countries = ['Tanzania','Argentina','Bahamas','Morocco','Canada','Colombia','Mexico','Marshall Islands']

In [208]:
dict_country[0]

KeyError: ignored

In [206]:
for i in range (len(x_val)):
  if x_val.iloc[i]['Country'] in Countries:
    print("here")
    lgb_pred[i]+=100000
  elif x_val.iloc[i]['Profession'] in Professions:
    lgb_pred[i]+=150000
  print(dict_country[x_val.iloc[i]['Country'])

22.0
4.0
4.0
9.0
4.0
9.0
35.0
56.0
22.0
8.0
44.0
35.0
34.0
63.0
77.0
9.0
18.0
43.0
4.0
27.0
109.0
4.0
45.0
6.0
80.0
39.0
17.0
44.0
4.0
8.0
64.0
33.0
28.0
18.0
28.0
6.0
46.0
13.0
54.0
42.0
19.0
17.0
32.0
50.0
42.0
30.0
42.0
21.0
15.0
76.0
58.0
59.0
16.0
102.0
10.0
21.0
4.0
61.0
24.0
49.0
49.0
13.0
56.0
25.0
28.0
60.0
4.0
76.0
16.0
60.0
87.0
59.0
22.0
46.0
40.0
72.0
34.0
8.0
25.0
31.0
4.0
52.0
50.0
19.0
60.0
4.0
64.0
34.0
4.0
33.0
4.0
34.0
4.0
12.0
52.0
50.0
4.0
26.0
36.0
18.0
12.0
13.0
15.0
41.0
33.0
12.0
60.0
52.0
96.0
28.0
4.0
25.0
21.0
35.0
10.0
39.0
73.0
9.0
69.0
59.0
45.0
10.0
54.0
4.0
58.0
25.0
48.0
4.0
48.0
35.0
64.0
18.0
17.0
63.0
40.0
68.0
34.0
57.0
43.0
56.0
11.0
16.0
10.0
16.0
53.0
39.0
27.0
7.0
4.0
41.0
21.0
32.0
25.0
30.0
12.0
4.0
25.0
32.0
7.0
40.0
24.0
68.0
12.0
35.0
30.0
119.0
25.0
52.0
4.0
85.0
26.0
24.0
76.0
35.0
4.0
19.0
4.0
31.0
14.0
56.0
73.0
5.0
26.0
102.0
4.0
4.0
16.0
64.0
41.0
4.0
72.0
19.0
6.0
31.0
39.0
22.0
4.0
48.0
13.0
16.0
28.0
27.0
81.0
30.0
30.0
24.0
74.0


KeyboardInterrupt: ignored

In [0]:
pred = x_val['Income']

In [204]:
mean_absolute_error(lgb_pred,y_val)

14396.914003103377

In [191]:
df2.groupby('Country')['diff'].mean().sort_values()[140:]

Country
Tanzania            145556.557669
Argentina           146464.890107
Bahamas             150723.609362
Morocco             156207.259933
Canada              163495.389462
Colombia            166811.777824
0                   169752.776202
Germany             178040.693400
Russia              189069.523010
Grenada             194491.298251
Kenya               238342.083900
Thailand            261133.140571
Marshall Islands    320583.922525
Mexico              719374.275745
Name: diff, dtype: float64